# Корневые эвристики

 Для начала вспомним, что такое структура:

In [0]:
struct pair{
    int first, second;
};
  
pair a;
cout << a.first << " " << a.second << "\n"; 

## Корневая на массиве

### Задача RSQ

Напомним вам данную задачу: дан массив $a$ длины $N$, также дано $M$ запросов одного из двух типов: 

1) Находить сумму на отрезке с $l$ по $r$.

2) Увеличивать все элементы на отрезке с $l$ по $r$ на $x$.

Вы наверняка все умеете решать данную задачу с помощью дерева отрезков, давайте теперь научимся ее решать при помощи корневой декомпозиции.

Давайте разделим весь массив на блоки по $\sqrt{N}$ и посчитаем сумму на каждом блоке. Так как блоков не больше $\frac{N}{\sqrt{N}} = \sqrt{N}$, то суммарно это работает за $\sqrt{N} ^2 = N$.

Теперь поймем, как отвечать на запросы:

1) Если блок целиком лежит в запросе, то просто возьмем сумму на нем, иначе все его элементы, которые лежат в запросе прибавим к полученному ответу.

2) Для блоков, целиком лежащих в запросе, просто прибавим к блоку $x * \sqrt{N}$  и запомним, что все элементы увеличины на $x$. Для оставшихся же элементов явно прибавим к ним и к ответу для их блока $x$.

Так как корень - функция вещественная, а также разделить ровно на $\sqrt{N}$ блоков может быть нельзя, то давайте брать число, близкое к корню, например при $N$ = 100000, как размер блока можно взять 320, так как 320 * 320 >= 100000


In [0]:
struct block {
    int sum, add;
};

int s = 320;
block dec[s];
for(int i = 0; i < n; i++) {
    dec[i / s].sum += a[i];
}
while(l < r) {
    if(l % s || l + s - 1 >= r) {
        ans += (a[l] + dec[l / s].add);
        l++;
    }
    else{
        ans += dec[l / s].sum;
        l += s;
    }
}

Оценим асимптотику этого алгоритма. Так как массив состоит из не более, чем $\sqrt{N}$ блоков и могут быть максимум два блока, не входящих в запрос полностью(слева и справа), то алгоритм работает за $O(M * \sqrt{N})$, где $M$ - число запросов, а $N$ - длина массива.

# Корневая на запросах

Давайте теперь разбивать на сам массив, а запросы к нему. Например, у нас есть такая структура, как префиксные суммы, которая позволяет находить сумму на отрезке за $O(1)$ без запросов обновления, мы можем честно пересчитывать ее каждый $\sqrt{M}$ запросов.

Теперь поговорим о самих запросах:

Так как мы пересчитываем префиксные суммы каждые $\sqrt{M}$ запросов, то мы не рассмотрели все запросы, лежащие после предыдущего обновления, таких запросов не может быть больше $\sqrt{M}$, то есть мы можем явно хранить список еще не обработанных запросов.

Тогда если мы встречаем запрос обновления, то просто добавим его в список, требующий обновления.

В случае запроса суммы просто возьмем ответ с помощью префиксных сумм и прибавим ответы на запросы.

In [0]:
struct q {
    int l, r, x;
};
int s = 320;
vector<q> was;
for(int i = 0; i < m; i++) {
    if(i % s == 0) {
        //пересчитываем префиксную сумму
        was.clear();
    }
		int t, l, r, x;
		cin >> t >> l >> r;
		if(t == 1) { //запрос суммы
        int ans = pi[r] - pi[l - 1];
        for(int j = 0; j < was.size(); j++) {
            ans += max(min(was[j].r, r) - max(was[j].l, l), 0) * was[j].x;
        }
		}
		else { //запрос обновления
         was.push_back({l, r, x});
		}
}

Так как в один запрос он может быть не более $\sqrt{M}$ неотвеченных запросов и каждые $\sqrt{M}$ мы пересчитываем наши префиксные суммы, то алгоритм работает за $O(N * \sqrt{M} + M * \sqrt{M})$

# Алгоритм Мо

Теперь обсудим еще одно решение этой же задачи. Допустим, что все запросы даны нам заранее и запросов обновления нет. Для каждого запроса закинем его в блок, где лежит его левая граница:

In [0]:
s_dec[l / s].a.push_back({l, r});


Затем внутри каждого блока отсортируем запросы по правой границе. Теперь давайте пройдемся по каждому блоку, поддерживая текущий отрезок, на который мы знаем ответ. И если текущий отрезок равен отрезку из запроса запоминать его, как ответ.

In [0]:
struct q {
    int l, r, index;
};

struct block {
    vector<q> a;
};

for(int i = 0; i < q; i++) {
    s_dec[l / s].a.push_back({l, r, i});
}

for(int i = 0; i < s; i++) {
    sort(all(s_dec[i].a), comp);
}

for(int i = 0; i < s; i++) {
    int l = i * s, r = i * s;
    int ans = 0;
    for(int j = 0; j < s_dec[i].a.size(); j++) {
        int tl = s_dec[i].a[j].l, tr = s_dec[i].a[j].r;
        while (r > tr){
            r—;
            ans -= a[r];
        }
        while(r < tr) {
            ans += a[r];
            r++;
        }
        while(l > tl) {
            l--;
            ans += a[l];
        }
        while(l < tl) {
            ans -= a[l];
            l++;
        }
        answ[s_dec[i].a[j].index] = ans;
}

Подумаем над асимптотикой данного алгоритма, для каждого запроса левая граница пройдет не более, чем длину блока, но при этом правая граница идет только вперед, а следовательно для одного блока пройдет не более, чем длину массива, то есть суммарно алгоритм работает за $O(N*\sqrt(M) + M*\sqrt(N))$.

Но зачем для такой легкой задачи использовать такой сложный алгоритм? Алгоритм Мо может помочь нам в ответе на гораздо более сложные вопросы - k-ая статистика на отрезке, медиана отрезка, наиболее встречающийся элемент на отрезке и так далее.

### Практическое задание

https://codeforces.com/problemset/problem/840/D?locale=ru

# Деление на тяжелые и легкие объекты

В математике очень часто встречается идея рассмотреть все объекты с каким-то свойством < $\sqrt{N}$ и больше.

Например всем известный алгоритм проверки на простоту чисел(рассмотрим делители < $\sqrt{N}$ и больше, но так как для любого делителя больше корня можно найти делитель меньше корня => нам надо честно рассмотреть только первые) или например факт, что если суммарная длина строк = $N$, то различных длин строк будет не более $\sqrt{N}$ или если вы увидели ограничение $A * B <= N$, то одно из чисел < $\sqrt{N}$. 

### Практическое задание

https://codeforces.com/group/5xexwZ9WG5/contest/792/problem/E

# Подбор константы

В корневой декомпозиции очень важен размер блока. Обычно брать s = $\sqrt{N}$ или близкое к ней число(желательно, чтобы делилось на максимально возможную степень двойки) оптимально, но сейчас мы поговорим об особых случаях. Пусть s - размер блока и мы знаем, что алгоритм работает за $O(N * s + \frac{N ^ 2 * \log(N)}{ s})$, тогда если взять s = $\sqrt{N} * 4$, то алгоритм будет работать быстрее, чем при s = $\sqrt{N}$, то есть при определении размера блока первым делом надо смотреть на асимптотику.

### Контест

https://informatics.msk.ru/moodle/mod/statements/view.php?id=33473#1

## Корзины

Оптимизация заключается в объеденении двух подходов. Теперь мы не просто делим массив на блоки, но еще и каждый корень запросов, пересчитываем блоки, это может быть полезно, если вы нам нужно вставлять/удалять из произвольного места в массиве и вычислять что-то на отрезках.